vllm

By default, vLLM downloads model from HuggingFace. If you would like to use models from ModelScope in the following examples, please set the environment variable:

`export VLLM_USE_MODELSCOPE=True`

In [1]:
from vllm import LLM, SamplingParams

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-11 10:40:27,491	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [8]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
    "今天天气真好，咱们出去",
    "明天就要开学了，我的作业还没写完，",
    "请你介绍一下你自己。AI："
]
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=2048)

In [11]:
SamplingParams?

Init signature:
SamplingParams(
    n: int = 1,
    best_of: Optional[int] = None,
    presence_penalty: float = 0.0,
    frequency_penalty: float = 0.0,
    repetition_penalty: float = 1.0,
    temperature: float = 1.0,
    top_p: float = 1.0,
    top_k: int = -1,
    min_p: float = 0.0,
    use_beam_search: bool = False,
    length_penalty: float = 1.0,
    early_stopping: Union[bool, str] = False,
    stop: Union[List[str], str, NoneType] = None,
    stop_token_ids: Optional[List[int]] = None,
    include_stop_str_in_output: bool = False,
    ignore_eos: bool = False,
    max_tokens: Optional[int] = 16,
    logprobs: Optional[int] = None,
    prompt_logprobs: Optional[int] = None,
    skip_special_tokens: bool = True,
    spaces_between_special_tokens: bool = True,
    logits_processors: Optional[List[Callable[[List[int], torch.Tensor], torch.Tensor]]] = None,
) -> None
Docstring:     
Sampling parameters for text generation.

Overall, we follow the sampling parameters from the Open

In [ ]:
model_path = 'ZhipuAI/chatglm3-6b'
llm = LLM(
    model=model_path,
    trust_remote_code=True,
    tokenizer=model_path,
    tokenizer_mode='slow',
    tensor_parallel_size=1
)

In [9]:
outputs = llm.generate(prompts, sampling_params)

Processed prompts: 100%|██████████| 6/6 [00:07<00:00,  1.23s/it]


In [10]:
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Prompt: 'Hello, my name is', Generated text: ' [Name], and I am an AI language model. I am here to provide you with the best possible assistance. How can I help you today?'
Prompt: 'The president of the United States is', Generated text: ' required to be a natural-born citizen of the United States.\n已成为美国总统的某些条件是：必须是美国公民的自然 born。'
Prompt: 'The capital of France is', Generated text: ' Paris.'
Prompt: 'The future of AI is', Generated text: ' bright, but also very uncertain. –> 5 min. video\nIt discusses the potential benefits and risks of AI, and the challenges of developing AI in a way that is safe, ethical, and beneficial for society. It also touches on the topic of AI and job displacement, as well as the potential for AI to improve healthcare and other areas. The speaker argues that while the future of AI is uncertain, it is important to be aware of the potential risks and work to mitigate them in order to ensure that AI is developed in a way that is safe and beneficial for society.'


In [12]:
from modelscope import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

2024-04-10 17:23:25,406 - modelscope - WARNING - Model revision not specified, use revision: v1.0.2


In [20]:
output.outputs[0].token_ids

[5954,
 30932,
 498,
 356,
 941,
 2787,
 2114,
 941,
 5589,
 30930,
 13,
 4529,
 284,
 11265,
 11476,
 30932]

In [21]:
tokenizer.decode(output.outputs[0].token_ids)

'bright, but with great potential comes great responsibility.\nAs an AI developer,'

## 对比不使用vllm的大模型推理速度

In [ ]:
from modelscope import snapshot_download
from modelscope import AutoTokenizer, AutoModel

model_name = "chatglm3-6b"
model_path = snapshot_download('ZhipuAI/chatglm3-6b')
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).half().cuda()
model = model.eval()

In [8]:
from tqdm import tqdm

In [ ]:
res = []

for q in tqdm(prompts):
    t = model.chat(tokenizer, q)
    res.append(t)

In [12]:
!pwd

/mnt/workspace/libs
